In [42]:
import requests as r
import json
import pandas as pd
import io
import regex as re
import csv
from requests import HTTPError
from dataclasses import dataclass, field
import time
from datetime import date, timedelta
import os

# Класс данных, которые мы хотим получить. По умолчанию временной отрезок это последний год (формат: "2023-03-28")
По желанию ненужные поля можно просто закомментировать

In [43]:
@dataclass
class RequestParams:
    start_date: str = date.today().replace(year=date.today().year - 1)
    end_date: str = date.today() - timedelta(days=1)
    fields: list[str] = field(default_factory=lambda: [
        'ym:s:visitID',             # Идентификатор визита
        'ym:s:clientID',            # Анонимный идентификатор пользователя в браузере (first-party cookies)
        'ym:s:date',                # Дата визита
        'ym:s:watchIDs',            # Просмотры, которые были в данном визите. Ограничение массива — 500 просмотров
        'ym:s:startURL',            # Страница входа
        'ym:s:endURL',              # Страница выхода
        'ym:s:pageViews',           # Глубина просмотра (детально)
        'ym:s:visitDuration',       # Время на сайте (детально)
        'ym:s:regionCountry',       # ID страны
        'ym:s:regionCity',          # ID города
        'ym:s:goalsID',             # Номера целей, достигнутых за данный визит
        'ym:s:goalsSerialNumber',   # Порядковые номера достижений цели с конкретным идентификатором
        'ym:s:referer',             # Реферер
        'ym:s:deviceCategory',      # Тип устройства. Возможные значения: 1 — десктоп, 2 — мобильные телефоны, 3 — планшеты, 4 — TV
        'ym:s:operatingSystem',     # Операционная система (детально)
        'ym:s:browser',             # Браузер
    ])
    source: str = 'visits'

# Получение готовых логов (Документация яндекс Logs API)

1.   Чтобы создать логи, выполните запрос POST https://api-metrika.yandex.net/management/v1/counter/{counterId}/logrequests. Вы можете оценить возможность выгрузки данных с помощью запроса GET https://api-metrika.yandex.net/management/v1/counter/{counterId}/logrequests/evaluate, если не уверены, что его удастся выгрузить.
2.   После успешного выполнения запроса на создание логов сохраните request_id и дождитесь подготовки лога. Узнать статус обработки лога можно с помощью запроса GET https://api-metrika.yandex.net/management/v1/counter/{counterId}/logrequest/{requestId}.
3. Лог со статусом processed готов к выгрузке. Для выгрузки используйте запрос GET https://api-metrika.yandex.net/management/v1/counter/{counterId}/logrequest/{requestId}/part/{partNumber}/download.

4. После выгрузки очистите подготовленные для загрузки логи, чтобы освободить место для следующих запросов. Для этого выполните запрос POST https://api-metrika.yandex.net/management/v1/counter/{counterId}/logrequest/{requestId}/clean.

# (В самом конце написаны вызовы запросов раздельно, если это необходимо)


In [44]:
# STEP 0 is checking possibility of request execution according demand parameters
def evaluate_request(counter_id : str, token : str, request_params : RequestParams) -> bool:
    url = f'https://api-metrika.yandex.net/management/v1/counter/{counter_id}/logrequests/evaluate'
    headers = {'Authorization': f'Bearer {token}'}
    fields = {
        'date1' : request_params.start_date,
        'date2' : request_params.end_date,
        'fields' : ','.join(request_params.fields),
        'source' : request_params.source
    }
    response = r.get(url, headers=headers, params=fields)
    
    if response.status_code != 200:
        print(response.text)
        # print(f"date1: {fields['date1']}, date2: {fields['date2']}")
        raise HTTPError(response.status_code)
    possible = response.json()['log_request_evaluation']['possible']
    if not possible:
        print(f'Request with params {request_params} could not be executed')
    return possible

In [45]:
# STEP 1 is creating log file according to your willingness.
# Additional request is provided as https://api-metrika.yandex.net/management/v1/counter/{counter_id}/logrequests/evaluate but anyway you would be notified if smth went wrong

# The output is request_id (int/string) according to parameters, which were provided
def create_logs(counter_id : str, token : str, request_params: RequestParams) -> str:
    url = f'https://api-metrika.yandex.net/management/v1/counter/{counter_id}/logrequests'
    headers = {'Authorization': f'Bearer {token}'}
    fields = {
        'date1' : request_params.start_date,
        'date2' : request_params.end_date,
        'fields' : ','.join(request_params.fields),
        'source' : request_params.source
    }
    response = r.post(url, headers=headers, params=fields)

    if response.status_code != 200:
        raise HTTPError(response.status_code)

    request_id = response.json()['log_request']['request_id']
    return request_id

In [46]:
# STEP 2 is getting amount of parts which your log was separated. 
# It could vary from 7 to 120 seconds

# The output is amount of parts which data was separated.
def get_parts_number(counter_id : str, token : str, request_id : str) -> int :
    url = f"https://api-metrika.yandex.net/management/v1/counter/{counter_id}/logrequest/{request_id}"
    headers = {'Authorization': f'Bearer {token}'}

    while True:
        response = r.get(url, headers=headers)

        if response.status_code != 200:
            raise HTTPError(response.status_code)
        if response.json()['log_request']['status'] == 'processed':
            break
        print(f"Request id: {request_id} Status: {response.json()['log_request']['status']}")
        time.sleep(5)

    print(f"Request id: {request_id} Status: {response.json()['log_request']['status']}")
    part_numbers = len(response.json()['log_request']['parts'])
    return part_numbers

In [47]:
def save_to_csv_file(tsv_data, counter_id : str, request_id : str, part : int) -> str:
    rows = tsv_data.split("\n")
    filename = f"output{counter_id}_{request_id}_{part}.csv"
    csv_writer = csv.writer(open(filename, "w", newline="", encoding="utf-8"))
    for row in rows:
        columns = row.split("\t")
        csv_writer.writerow(columns)
    print(f"File {filename} created")
    return filename

In [48]:
# STEP 3 is getting all this parts. 

# Provided data has TSV format, thus we convert and save this data in csv files and altogether.
def download_data(counter_id : str, token : str, request_id : str, parts : int, clean_partial_files : bool=True) -> None :
    headers = {'Authorization': f'Bearer {token}'}
    filenames = []
    for part_number in range(parts):
        url = f"https://api-metrika.yandex.net/management/v1/counter/{counter_id}/logrequest/{request_id}/part/{part_number}/download"
        response = r.get(url, headers=headers)
        if response.status_code != 200:
            raise HTTPError(response.status_code)
        filename = save_to_csv_file(response.text, counter_id, request_id, part_number)
        filenames.append(filename)
        
    dfs = []
    for filename in filenames:
        dfs.append(pd.read_csv(filename))
    merged_df = pd.concat(dfs, ignore_index=True)
    
    filename_all = f'output{counter_id}_{request_id}_all.csv'
    merged_df.to_csv(filename_all, index=False)
    print(f"File {filename_all} saved")
    
    if clean_partial_files:
        for filename in filenames:
            os.remove(filename)
            print(f"File {filename} was removed")

In [49]:
# STEP 4 is cleaning log file if necessary
def clean_logfile(counter_id : str, token : str, request_id : str) -> None:
    url = f"https://api-metrika.yandex.net/management/v1/counter/{counter_id}/logrequest/{request_id}/clean"
    headers = {'Authorization': f'Bearer {token}'}
    response = r.post(url, headers=headers)
    if response.status_code != 200:
        raise HTTPError(response.status_code)
    print(f"Log request: {request_id} was successfully removed")

In [50]:
# COUNTER_ID and TOKEN must be written manually for security purposes 
def main(counter_id : str, token : str, request_params : RequestParams, clean_log=False, clean_partial_files=True) -> None:
    possible = evaluate_request(counter_id, token, request_params)
    if not possible:
        return
    request_id = create_logs(counter_id, token, request_params)
    part_numbers =  get_parts_number(counter_id, token, request_id)
    download_data(counter_id, token, request_id, part_numbers, clean_partial_files)
    if clean_log:
        clean_logfile(counter_id, token, request_id)

# Счетчик и токен решил не хранить здесь. Необходимо их сначала ввести в ячейку ниже

In [51]:
COUNTER_ID = ""
TOKEN = ""
REQUEST_PARAMS = RequestParams()

# Немного о входных данных
* **(ШАГ 3)** Алгоритм сохранения данных предполагает предварительное сохранение данных в 
каждый файл на случай если не хватит памяти соеденить все части за раз, поэтому можно оставить файлы, на которые были разбиты данные (параметр **clean_partial_files=True** по умолчанию)

* **(ШАГ 3)** Так как можно получить данные из любого существующего лога со статусом "processed" (чтобы 2 минуты не ждать каждый раз для одного и того же запроса), то можно оставить лог неудаленным (**clean_log=True** по умолчанию). 


In [52]:
main(counter_id=COUNTER_ID, 
     token=TOKEN, 
     request_params=REQUEST_PARAMS, 
     clean_log=True, 
     clean_partial_files=True)

{
  "errors" : [ {
    "error_type" : "unauthorized",
    "message" : "Неавторизованный пользователь"
  } ],
  "code" : 401,
  "message" : "Неавторизованный пользователь"
}


HTTPError: 401

# Шаг 1. Проверка

In [53]:
evaluate_request(counter_id=COUNTER_ID,
                 token=TOKEN, 
                 request_params=REQUEST_PARAMS)

{
  "errors" : [ {
    "error_type" : "unauthorized",
    "message" : "Неавторизованный пользователь"
  } ],
  "code" : 401,
  "message" : "Неавторизованный пользователь"
}


HTTPError: 401

# Шаг 1. Создание лога

In [54]:
request_id_test = create_logs(counter_id=COUNTER_ID, 
                         token=TOKEN, 
                         request_params=REQUEST_PARAMS)

HTTPError: 401

# Шаг 2. Получение количества частей, на которые разбиты данные

In [55]:
part_numbers_test =  get_parts_number(counter_id=COUNTER_ID, 
                                 token=TOKEN, 
                                 request_id=request_id_test)

NameError: name 'request_id_test' is not defined

# Шаг 3. Скачать данные

In [56]:
download_data(counter_id=COUNTER_ID, 
              token=TOKEN, 
              request_id=request_id_test, 
              parts=part_numbers_test,
              clean_partial_files=True)

NameError: name 'request_id_test' is not defined

# Шаг 4. Удалить лог из Яндекс Метрики

In [57]:
clean_logfile(counter_id=COUNTER_ID, 
              token=TOKEN, 
              request_id=request_id_test)

NameError: name 'request_id_test' is not defined